Conditional probability in two dimensions

$$ P(A|B) = \frac{P(B \cap A)}{P(B)} $$

$$ \Rightarrow P(A \cap B) = P(A|B)P(B) = P(B|A)P(A) $$

Conditional probability in 3 dimensions

Treat $B$ as $B \cap C$

$$ P(A|B \cap C) = \frac{P(A \cap B \cap C)}{P(B \cap C)} $$


$$ \Rightarrow P\left(A \cap (B \cap C) \right) \\
= P(A|B \cap C)P(B \cap C) \\
= P(B|A \cap C)P(A \cap C) \\
= P(C|A \cap B)P(A \cap B) \\
= P(B \cap C|A)P(A) $$

$$ P(w \cap x \cap y) $$


$$ P(w | y \cap x) = \frac{P(w \cap x \cap y)}{P(y \cap x)}
= \frac{P(w \cap x \cap y)}{P(y|x)P(x)}
= \frac{P(y|x \cap w)P(x \cap w)}{P(y|x)P(x)}
$$

$$ = \frac{P(y|x \cap w)P(x|w)P(w)}{P(y|x)P(x)}
= \frac{P(y|x \cap w)P(w)}{P(y|x)}
$$